In this example we solve a scalar *unfitted* PDE problem. As a
discretisation method we use a level set based geometry description and
a Cut (or Fictitious) Finite element method with a Nitsche formulation
to impose boundary conditions. For stability we add a ghost penalty
stabilization. See also https://github.com/ngsxfem/ngsxfem/blob/release/demos/fictdom.py

## Imports

In [ ]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *

## basic parameters

In [ ]:
# Mesh diameter
maxh = 0.2
# Finite element space order
order = 1
# Stabilization parameter for ghost-penalty
gamma_stab = 0.1
# Stabilization parameter for Nitsche
lambda_nitsche = 10 * order * order

## background mesh

In [ ]:
# Geometry and Mesh
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)

## exact levelset and data

In [ ]:
r = sqrt(x**2 + y**2)
levelset = r-1

exact = sin(x+y)
coeff_f = - (exact.Diff(x).Diff(x) + exact.Diff(y).Diff(y)).Compile()

## level set approximation
The level set function has to be approximated to allow for realizable quadrature.

In [ ]:
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)

## Marking the active part of the mesh (and the cut elements)

In [ ]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)

## Marking all facets used to stabilize with ghost penalties

In [ ]:
# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)

## Restricted FESpace

In [ ]:
Vhbase = H1(mesh, order=order, dirichlet=[], dgjumps=True)
Vh = Restrict(Vhbase, hasneg)
gfu = GridFunction(Vh)

## Symbols for the definition of the discrete variational formulation

In [ ]:
u, v = Vh.TrialFunction(), Vh.TestFunction()
h = specialcf.mesh_size
n = Normalize(grad(lsetp1))

# integration domains:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)
dw = dFacetPatch(definedonelements=ba_facets)

## the discrete variational formulation

In [ ]:
a = BilinearForm(Vh, symmetric=True)
# Diffusion term
a += grad(u) * grad(v) * dx
# Nitsche term
a += -grad(u) * n * v * ds
a += -grad(v) * n * u * ds
a += (lambda_nitsche / h) * u * v * ds
# Ghost penalty stabilization (near the boundary)
a += gamma_stab / h**2 * (u - u.Other()) * (v - v.Other()) * dw

# R.h.s. term:
f = LinearForm(Vh)
f += coeff_f * v * dx
f += exact * (-grad(v) * n + (lambda_nitsche / h) * v) * ds

## Setup and solution of linear systems

In [ ]:
# Assemble system
a.Assemble()
f.Assemble()

# Solve linear system
gfu.vec.data = a.mat.Inverse(Vh.FreeDofs(),inverse="sparsecholesky") * f.vec

## Error evaluation

In [ ]:
# Measure the error
l2error = sqrt(Integrate((gfu - exact)**2*dx, mesh))
print("L2 Error: {0}".format(l2error))

## Visualization of the solution

In [ ]:
# visualization:
DrawDC(lsetp1, gfu, 0, mesh, "uh")
